# Installing Prometheus

We will download the current stable version locally and just start the binary,  

In [ ]:
WORKDIR="/tmp/lfs-prometheus"

PROM_VERSION="2.49.1"
DEMO_VERSION="0.11.1"
NODEEXP_VERSION="1.7.0"
CADVISOR_VERSION="0.36.0"

In [ ]:
%%bash -s {WORKDIR}

{
    echo "Cloning the repo"
    WORKDIR=$1
    if [ ! -e ${WORKDIR}/.git ]; then 
        git clone https://github.com/ennc0d3-learn/lfs-prometheus ${WORKDIR} 
    else 
        cd ${WORKDIR} && git pull --rebase
        ls -l ${WORKDIR}
    fi
    echo "Ready to go!"
}

In [ ]:
%%bash -s  {WORKDIR} {PROM_VERSION}
(
    WORKDIR=$1
    PROM_VERSION=$2
    cd ${WORKDIR}
    wget -q https://github.com/prometheus/prometheus/releases/download/v${PROM_VERSION}/prometheus-${PROM_VERSION}.linux-amd64.tar.gz
    tar -zxvf prometheus-${PROM_VERSION}.*.gz
    
)


In [ ]:
%%bash -s {WORKDIR} {PROM_VERSION}
(
    WORKDIR=$1
    PROM_VERSION=$2
    prometheus_dir=${WORKDIR}/prometheus-${PROM_VERSION}*
    cat > ${prometheus_dir}/prometheus.yml <<-EOD
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']
EOD
echo "Created the prometheus.yml"
cat ${prometheus_dir}/prometheus.yml
)

In [ ]:
%%bash -s {WORKDIR}
(
    WORKDIR=$1
    PROM_VERSION=$2
    cd $WORKDIR/prometheus-*/
    killall prometheus
    ./prometheus &
    
)

Download the prepare the demo service 

In [ ]:
%%bash -s {WORKDIR} {DEMO_VERSION}
(
    WORKDIR=$1
    DEMO_VERSION=${2:-0.11.1}
    cd $WORKDIR
    echo "Downloading the demo service version ${DEMO_VERSION}"
    wget -q https://github.com/juliusv/prometheus_demo_service/releases/download/${DEMO_VERSION}/prometheus_demo_service-${DEMO_VERSION}.linux-amd64
    
    chmod +x ./prometheus_demo_service-${DEMO_VERSION}.linux-amd64
    mv ./prometheus_demo_service-${DEMO_VERSION}.linux-amd64 ./prometheus_demo_service
    
    echo "Starting the demo service instances"
    
    killall ./prometheus_demo_service
    
    ./prometheus_demo_service -listen-address=":10001" & > /dev/null
    ./prometheus_demo_service -listen-address=":10002" & > /dev/null
    ./prometheus_demo_service -listen-address=":10003" & > /dev/null
    
    echo "Demo instances are running"
    
)

In [ ]:
%%bash -s {WORKDIR}
(
    WORKDIR=$1
    cat > ${WORKDIR}/prometheus-*/prometheus.yml <<-EOD 
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']
    - job_name: 'demo-service'
      static_configs:
        - targets:
            - localhost:10001
            - localhost:10002
            - localhost:10003
EOD
    
    # Send signal to reaload
    killall -HUP prometheus
)



## Installing node exporter

In [ ]:
%%bash -s {WORKDIR} {NODEEXP_VERSION}
{
    cleanup() {
        trap - INT TERM
        echo "Cleaning up the node_exporter"
        killall node_exporter
        exit 1
    }
    
    trap cleanup INT TERM
    
    WORKDIR=$1
    NODEEXP_VERSION=${2:-1.7.0}
    startNodeExporter() {
        cd $WORKDIR
        wget -q https://github.com/prometheus/node_exporter/releases/download/v${NODEEXP_VERSION}/node_exporter-${NODEEXP_VERSION}.linux-amd64.tar.gz -O node_exporter.tgz
        tar -zxf node_exporter.tgz
        chmod +x ./node_exporter*/node_exporter
        killall node_exporter
        ./node_exporter*/node_exporter & > /dev/null
    }
    echo "Download and start the exporter"
    startNodeExporter
}

Update the scrape targets to include the node_exporter service running on 9100

In [ ]:
%%bash -s {WORKDIR}
(
    WORKDIR=$1
    cat > ${WORKDIR}/prometheus-*/prometheus.yml <<-EOD 
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']
    - job_name: 'demo-service'
      static_configs:
        - targets:
            - localhost:10001
            - localhost:10002
            - localhost:10003
    - job_name: 'node_exporter'
      static_configs:
        - targets:
            - localhost:9100
EOD
    
    # Send signal to reaload
    killall -HUP prometheus
)


Install/Run cAdvisor

In [ ]:
%%bash -s {WORKDIR} {CADVISOR_VERSION}
{
    WORKDIR=$1
    CADVISOR_VERSION=${2:-"0.36.0"}
    
    cleanup() {
        trap - INT TERM
        echo "Stopping cAdvisor"
        docker rm -f cadvisor
        exit 1
    }
    
    startCadvisor() {
        docker rm -f cadvisor
        docker run \
        --volume=/:/rootfs:ro \
        --volume=/var/run:/var/run:ro \
        --volume=/sys:/sys:ro \
        --volume=/var/lib/docker/:/var/lib/docker:ro \
        --volume=/dev/disk/:/dev/disk:ro \
        --publish=8080:8080 \
        --detach=true \
        --name=cadvisor \
        --privileged \
        --device=/dev/kmsg \
        gcr.io/cadvisor/cadvisor:v${CADVISOR_VERSION}
    }
    
    trap cleanup INT TERM
    startCadvisor
}

In [ ]:
%%bash -s {WORKDIR}
(
    WORKDIR=$1
    cat > ${WORKDIR}/prometheus-*/prometheus.yml <<-EOD 
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']
    - job_name: 'demo-service'
      static_configs:
        - targets:
            - localhost:10001
            - localhost:10002
            - localhost:10003
    - job_name: 'node_exporter'
      static_configs:
        - targets:
            - localhost:9100
    - job_name: 'cadvisor'
      static_configs:
        - targets:
            - localhost:8080
EOD
    
    # Send signal to reaload
    killall -HUP prometheus
)


Node Exporter - Custom 

In this chapter, we create an simple Node Exporter (CPU metrics collection), the program is written in python [cpu-metrics-exporter.py](./cpu-metrics-exporter/cpu-metric-exporter.py)

It uses psutil and get the cpu data and use CounterMetricFamily to transform to prometheus format.

In [ ]:
%%bash -s {WORKDIR}
{
    WORKDIR=$1
    cd $WORKDIR
    killall cpu-metric-exporter.py
    python cpu-metrics-exporter/cpu-metric-exporter.py
}


In [ ]:
%%bash -s {WORKDIR}
(
    WORKDIR=$1
    cat > ${WORKDIR}/prometheus-*/prometheus.yml <<-EOD 
global:
    scrape_interval: 5s
    evaluation_interval: 5s
scrape_configs:
    - job_name: 'prometheus'
      static_configs:
        - targets: ['localhost:9090']
    - job_name: 'demo-service'
      static_configs:
        - targets:
            - localhost:10001
            - localhost:10002
            - localhost:10003
    - job_name: 'node_exporter'
      static_configs:
        - targets:
            - localhost:9100
    - job_name: 'cadvisor'
      static_configs:
        - targets:
            - localhost:8080
    - job_name: 'cpu-metrics'
      static_configs:
        - targets:
            - localhost:8100
EOD
    # Send signal to reaload
    killall -HUP prometheus
)